<a href="https://colab.research.google.com/github/mgersins-design/Machine-Learning/blob/main/HW6/FCNN_CIFAR_10_small.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [2]:
transform_process = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.4914, 0.4822, 0.4465],
        std=[0.2470, 0.2435, 0.2616]
    )
])

data_path = './data'

cifar10_train = datasets.CIFAR10(
    data_path, train=True, download=True, transform=transform_process
)
cifar10_val = datasets.CIFAR10(
    data_path, train=False, download=True, transform=transform_process
)

batch_size = 64
train_loader = DataLoader(cifar10_train, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(cifar10_val, batch_size=batch_size, shuffle=False)

print(f"Trainingsdaten: {len(cifar10_train)} Bilder")
print(f"Validierungsdaten: {len(cifar10_val)} Bilder")

100%|██████████| 170M/170M [00:14<00:00, 11.9MB/s]


Trainingsdaten: 50000 Bilder
Validierungsdaten: 10000 Bilder


In [3]:
from torch.nn.modules.activation import LogSoftmax
import torch
import torch.nn as nn

model = nn.Sequential(
    nn.Linear(3 * 32 * 32, 512),
    nn.ReLU(),
    nn.Linear(512, 10)
)

learning_rate = 1e-2
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
loss_fn = nn.CrossEntropyLoss ()
n_epochs = 20

In [4]:
import time
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for epochs in range(n_epochs):
  epoch_start_time = time.time()

  # --- Training Phase ---
  model.train()
  for imgs, labels in train_loader:

    imgs = imgs.to(device)
    labels = labels.to(device)

    batch_size = imgs.shape[0]
    out = model(imgs.view(batch_size, -1))
    loss = loss_fn(out, labels)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

  # --- Validation Phase ---
  model.eval()
  correct = 0
  total = 0

  with torch.no_grad():
    for imgs, labels in val_loader:
      imgs = imgs.to(device)
      labels = labels.to(device)

      outputs = model(imgs.view(imgs.shape[0], -1))
      _, predicted = torch.max(outputs, dim=1)
      total += labels.shape[0]
      correct += int((predicted == labels).sum())

      accuracy = correct/total

  epoch_end_time = time.time()
  epoch_time = epoch_end_time - epoch_start_time

  print(f"Epoch: {epochs+1:2d}, Time: {epoch_time:.2f}s, Training Loss: {loss.item():.4f}, Validation Accuracy: {accuracy:.4f}")

Epoch:  1, Time: 14.82s, Training Loss: 1.8143, Validation Accuracy: 0.4448
Epoch:  2, Time: 14.19s, Training Loss: 1.4686, Validation Accuracy: 0.4708
Epoch:  3, Time: 14.25s, Training Loss: 1.1239, Validation Accuracy: 0.4857
Epoch:  4, Time: 13.97s, Training Loss: 1.4736, Validation Accuracy: 0.4910
Epoch:  5, Time: 14.07s, Training Loss: 1.3378, Validation Accuracy: 0.5048
Epoch:  6, Time: 14.09s, Training Loss: 0.9354, Validation Accuracy: 0.5149
Epoch:  7, Time: 14.18s, Training Loss: 0.8728, Validation Accuracy: 0.5129
Epoch:  8, Time: 14.12s, Training Loss: 1.8637, Validation Accuracy: 0.4873
Epoch:  9, Time: 14.05s, Training Loss: 0.9591, Validation Accuracy: 0.5210
Epoch: 10, Time: 13.95s, Training Loss: 1.1721, Validation Accuracy: 0.5118
Epoch: 11, Time: 14.18s, Training Loss: 0.7099, Validation Accuracy: 0.5168
Epoch: 12, Time: 14.10s, Training Loss: 1.2439, Validation Accuracy: 0.5073
Epoch: 13, Time: 14.16s, Training Loss: 1.4823, Validation Accuracy: 0.5057
Epoch: 14, T